In [19]:
import json
import itertools
import pandas as pd
import numpy as np
import os

In [20]:
# -------------------------------------------
# Load preprocessed data
# -------------------------------------------

with open("build/sections.json") as f:
    sections = json.load(f)

with open("build/slots.json") as f:
    slots = json.load(f)

print("Sections:", len(sections))
print("Slots:", len(slots))


Sections: 131
Slots: 209


In [21]:
# -------------------------------------------
# Assign each (course,sec) a binary variable index
# -------------------------------------------

sec_keys = [(s["course"], s["sec"]) for s in sections]
var_index = { key:i for i,key in enumerate(sec_keys) }
num_vars = len(var_index)

print("QUBO variables:", num_vars)

# QUBO stored in sparse 3-column dict
Q = {}   # {(i,j): value}

def add_Q(i, j, val):
    """Add val to Q[(i,j)], symmetric."""
    if (i,j) not in Q:
        Q[(i,j)] = 0
    Q[(i,j)] += val

QUBO variables: 115


In [22]:
# -------------------------------------------
# 1. One section per course
# Sum(x_section) = 1 → penalize choosing 0 or >1
# -------------------------------------------

lambda_course = 4.0

courses = {}
for (c,s) in var_index.keys():
    courses.setdefault(c, []).append(var_index[(c,s)])

for c, vars_c in courses.items():
    # quadratic penalty
    for i,j in itertools.combinations(vars_c, 2):
        add_Q(i, j, lambda_course)
    # linear penalty
    for i in vars_c:
        add_Q(i, i, -lambda_course)

In [23]:
# -------------------------------------------
# Build a quick lookup for time slots
# -------------------------------------------

slot_lookup = {}  # key: (course,sec) → list of time slots
for s in slots:
    key = (s["course"], s["sec"])
    slot_lookup.setdefault(key, []).append(s)

In [24]:
# -------------------------------------------
# Helper: check time overlap
# -------------------------------------------

def overlaps(a, b):
    return a["day"] == b["day"] and not (a["end"] <= b["start"] or b["end"] <= a["start"])


In [25]:
# -------------------------------------------
# 2. Instructor conflict
# -------------------------------------------

lambda_instr = 4.0

instr_groups = {}
for s in sections:
    key = (s["course"], s["sec"])
    instr_groups.setdefault(s["instructor"], []).append(key)

for instr, assignable in instr_groups.items():
    for a, b in itertools.combinations(assignable, 2):
        ia, ib = var_index[a], var_index[b]
        # check if they overlap in time
        for slotA in slot_lookup[a]:
            for slotB in slot_lookup[b]:
                if overlaps(slotA, slotB):
                    add_Q(ia, ib, lambda_instr)

In [26]:
# -------------------------------------------
# 3. Room conflict
# -------------------------------------------

lambda_room = 4.0

room_groups = {}
for s in sections:
    key = (s["course"], s["sec"])
    room_groups.setdefault(s["room"], []).append(key)

for room, assignable in room_groups.items():
    for a, b in itertools.combinations(assignable, 2):
        ia, ib = var_index[a], var_index[b]
        for slotA in slot_lookup[a]:
            for slotB in slot_lookup[b]:
                if overlaps(slotA, slotB):
                    add_Q(ia, ib, lambda_room)

In [27]:
# -------------------------------------------
# Save sparse QUBO as 3-column CSV
# -------------------------------------------

os.makedirs("build", exist_ok=True)
rows = []

for (i,j), v in Q.items():
    rows.append([i, j, float(v)])

qubo_df = pd.DataFrame(rows, columns=["i","j","value"])
qubo_df.to_csv("build/qubo_sparse.csv", index=False)

print("Saved sparse QUBO → build/qubo_sparse.csv")
print("Nonzero terms:", len(qubo_df))

Saved sparse QUBO → build/qubo_sparse.csv
Nonzero terms: 239
